In [1]:
%load_ext autoreload
%autoreload 2
#%pylab inline

In [2]:
import os
os.environ['KERAS_BACKEND'] = 'theano'
os.environ['THEANO_FLAGS'] = 'device=cpu'

In [3]:
import tensorflow as tf
import numpy as np
import itertools
import keras
import gym

from tqdm import tqdm
from wrappers.envs import AtariStackFrames
from utils.ae import AEConvKeras

Couldn't import dot_parser, loading of dot files will not be possible.


Using Theano backend.


In [4]:
model = keras.models.load_model('rllab_exp/model_epoch29.h5')
ar = np.random.rand(1,12,105,80)

In [5]:
m = keras.models.Model(input=model.layers[0].input, output=model.layers[-3].output)
print m.predict(ar)

[[  4.61665541e-01  -9.40143824e-01   3.95007968e+00  -4.76870984e-01
   -9.83308375e-01  -9.44032669e-01   2.96775985e+00   6.20017576e+00
    1.45723033e+00  -7.76356578e-01   4.38904715e+00  -9.49967146e-01
    2.84722495e+00   5.95845556e+00   1.93258190e+00   5.98257720e-01
   -7.31766939e-01   2.94834781e+00   4.20856810e+00  -9.70527530e-01
   -6.95844710e-01   3.72295833e+00   2.25248289e+00   2.94316459e+00
    6.21056700e+00  -9.03794885e-01  -8.98619294e-01  -8.95368516e-01
    7.73951387e+00   7.97684908e-01   8.73640776e-02  -9.94423449e-01
   -9.56115723e-01   8.12862873e+00   1.58732986e+00   6.86167049e+00
   -7.84784317e-01  -9.89890575e-01  -2.76907027e-01  -9.89010811e-01
   -1.92260012e-01  -9.99927223e-01   1.74333051e-01  -9.78452921e-01
    4.31194544e+00   1.12885296e+00  -9.83571291e-01  -9.92488325e-01
    4.77532768e+00  -8.70029926e-01  -8.49846244e-01   6.63873494e-01
   -4.91146028e-01  -9.99254167e-01   1.70630717e+00  -1.28747206e-02
   -9.99996126e-01  

In [ ]:
m.predict(ar).shape

In [ ]:
env = AtariStackFrames(gym.make('MsPacman-v0'), rgb=True)

In [ ]:
def collect_data(env=None, n_episodes=10, max_steps=5000):
    states = []
    for i in tqdm(range(n_episodes)):
        state = env.reset()
        states.append(state)
        terminal = False
        step = 0
        while not terminal and step < max_steps:
            action = env.action_space.sample()
            state, reward, terminal, _ = env.step(action)
            states.append(state)
            step += 1
    return np.array(states)

def run_episode(env=None, agent=None, max_steps=5000):
    state = env.reset()
    total_reward = 0.
    terminal = False
    step = 0
    while not terminal and step < max_steps:
        action = agent.predict(np.expand_dims(state, 0)).argmax(axis=1)
        #action = env.action_space.sample()
        state, reward, terminal, _ = env.step(action)
        total_reward += reward
        step += 1
    return total_reward

In [ ]:
res = []
for i in xrange(20):
    r = run_episode(env, model)
    print r
    res.append(r)
print 'mean: {:.3f}'.format(np.array(res).mean())

## temp

In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [ ]:
import keras
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model

In [ ]:
inp = Input(shape=(1,))
out = Dense(32, activation='relu')(inp)
out = Dense(1)(out)

In [ ]:
with tf.device('/job:ps/task:0'):
    print m.weights[0].device

In [ ]:
print model.input
print model.output

In [ ]:
model = Model(input=inp, output=out)

In [ ]:
model.save('temp.h5')

In [ ]:
m = keras.models.load_model('temp.h5')

In [ ]:
m.weights

In [ ]:
inp = tf.placeholder(tf.float32, shape=(None, 1))

In [ ]:
with tf.device('/job:ps/task:0'):
    #m = keras.models.load_model('temp.h5')
    mm = m(inp)

In [ ]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
# for var in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
#     print var.op.name

In [ ]:
tf.gradients(mm, m.weights[-1])

In [ ]:
with tf.device('/job:localhost/task:0'):
    m = keras.models.load_model('temp.h5')

In [ ]:
with tf.variable_scope('policy11'):
    m = keras.models.load_model('temp.h5')
    for var in m.weights:
        print var.op.name

In [ ]:
m.summary()

In [ ]:
m.layers[-2].output

In [ ]:
m.input

In [ ]:
with tf.variable_scope('policy'):
    out = Dense(128)(m.layers[-2].output)
    new = Model(input=m.layers[0].input, output=out)

In [ ]:
new.summary()

In [ ]:
for var in new.weights:
    print var.op.name

In [ ]:
m.layers[0].input